In [6]:
import re
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn import train_test

In [7]:
def vectorize(fen):
    data = re.split(" ", fen)
    rows= re.split("/", data[0])
    turn = data[1]
    can_castle = data[2]
    passant = data[3]
    half_moves = data[4]
    full_moves = data[5]
    
    bit_vector = np.zeros((13, 8, 8), dtype=np.uint8)
    #what layer each piece is found on
    piece_to_layer = {
            'R': 1,
            'N': 2,
            'B': 3,
            'Q': 4,
            'K': 5,
            'P': 6,
            'p': 7,
            'k': 8,
            'q': 9,
            'b': 10,
            'n': 11,
            'r': 12
        }
    #find each piece based on type
    for r,value in enumerate(rows):
        colum = 0
        for piece in value:
            if piece in piece_to_layer:
                bit_vector[piece_to_layer[piece],r,colum] =1
                colum += 1
            else:
                colum += int(piece)
    
    if turn.lower() == 'w':
        bit_vector [0,7,4] =1
    else:
        bit_vector [0,0,4] =1
        
    #where each castle bit is located
    castle ={
        'k': (0,0),
        'q': (0,7),
        'K': (7,0),
        'Q': (7,7),
        }

    for value in can_castle:
        if value in castle:
            bit_vector[0,castle[value][0],castle[value][1]] = 1
    
    #put en-passant square in the vector
    if passant != '-':
        bit_vector[0,  5 if (int(passant[1])-1 == 3) else 2 , ord(passant[0]) - 97,] = 1
        
    return bit_vector

In [8]:
print(vectorize(r'rnbqkb1r/ppp1pppp/3p4/3nP3/3P4/8/PPPK1PPP/RNBQ1BNR b kq - 1 4')[0])
# print(vectorize(r'r3r1k1/pppq2bp/3pp1p1/3P4/2PpP3/6P1/PP1N2K1/R1BQR3 b - e3 0 19')[0])

[[1 0 0 0 1 0 0 1]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0]]


In [9]:
def train_neural_network(df):
    # Define the neural network architecture
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(13, 8, 8)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])

    # Compile the model
    model.compile(optimizer='rmsprop',
                  loss='mean_squared_error',
                  metrics=['accuracy'])

    # Set training and testing data
    x_train, x_test, y_train, y_test = train_test_split(df.FEN, df.Evaluations, random_state=0, train_size = .75)

    # Train the model
    model.fit(x_train, y_train, epochs=10, batch_size=1)

    # Evaluate the model on the test data
    loss, accuracy = model.evaluate(x_test, y_test)

    # Print the evaluation metrics
    print(f"Test loss: {loss:.4f}")
    print(f"Test accuracy: {accuracy:.4f}")

In [10]:
df = pd.read_csv('../data/kaggleDataset/chessData.csv')

In [16]:
dfcopy = df.copy().head(1000)

In [17]:
vectorized = dfcopy.FEN.apply(vectorize)
dfcopy.FEN = vectorized
dfcopy

,FEN,Evaluation
0,"[[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",-10
1,"[[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",+56
2,"[[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",-9
3,"[[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",+52
4,"[[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",-26
...,...,...
995,"[[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",+20
996,"[[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",+135
997,"[[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",+84
998,"[[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...",+116


In [30]:
df.FEN[1:1000]

1      rnbqkbnr/pppp1ppp/4p3/8/4P3/8/PPPP1PPP/RNBQKBN...
2      rnbqkbnr/pppp1ppp/4p3/8/3PP3/8/PPP2PPP/RNBQKBN...
3      rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPP2PPP/RNBQKB...
4      rnbqkbnr/ppp2ppp/4p3/3p4/3PP3/8/PPPN1PPP/R1BQK...
5      rnbqkb1r/ppp2ppp/4pn2/3p4/3PP3/8/PPPN1PPP/R1BQ...
                             ...                        
995    r1bqk1nr/pp1nppbp/2pp2p1/3P2B1/4PP2/2N2N2/PPPQ...
996    r1bqk1nr/pp2ppbp/2pp2p1/2nP2B1/4PP2/2N2N2/PPPQ...
997    r1bqk1nr/pp2ppbp/2pp2p1/2nP2B1/4PP2/2N2N2/PPPQ...
998    r1b1k1nr/pp2ppbp/1qpp2p1/2nP2B1/4PP2/2N2N2/PPP...
999    r1b1k1nr/pp2ppbp/1qpp2p1/2nPP1B1/5P2/2N2N2/PPP...
Name: FEN, Length: 999, dtype: object

In [28]:
vectorized

0      [[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
1      [[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
2      [[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
3      [[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
4      [[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
                             ...                        
995    [[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
996    [[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
997    [[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
998    [[[1, 0, 0, 0, 0, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
999    [[[1, 0, 0, 0, 1, 0, 0, 1], [0, 0, 0, 0, 0, 0,...
Name: FEN, Length: 1000, dtype: object

In [21]:
dfcopy.to_csv('../data/createdData/vectorized.csv')

In [10]:
train_neural_network(dfcopy)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).